In [ ]:
import torch
import matplotlib.pyplot as plt
import random

# Callbacks

## Callbacks as GUI events

In [ ]:
import ipywidgets as widgets

From the ipywidget docs:
   - *the button widget is used to handle mouse clicks. The on_click method of the Button can be used to register function to be called when the button is clicked*

In [ ]:
w = widgets.Button(description='Click me')

In [ ]:
w

Button(description='Click me', style=ButtonStyle())

hi
hi
hi


In [ ]:
def f(o): print('hi')

In [ ]:
w.on_click(f)

NB: When callbacks are used in this way they are often called as "events"

## Creating your own callback 

In [ ]:
from time import sleep

In [ ]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [ ]:
slow_calculation()

30

In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb: cb(i)
    return res

In [ ]:
def show_progress(epoch): print(f"Awesome! We have finished epoch {epoch}!")

In [ ]:
slow_calculation(show_progress)

Awesome! We have finished epoch 0!
Awesome! We have finished epoch 1!
Awesome! We have finished epoch 2!
Awesome! We have finished epoch 3!
Awesome! We have finished epoch 4!


30

## Lambda and partials

In [ ]:
slow_calculation(lambda o: print(f"Awesome! We have finished epoch {o}!"))

Awesome! We have finished epoch 0!
Awesome! We have finished epoch 1!
Awesome! We have finished epoch 2!
Awesome! We have finished epoch 3!
Awesome! We have finished epoch 4!


30

In [ ]:
def show_progress(exclamation, epoch): print(f"{exclamation}! We've finished epoch {epoch}!")

In [ ]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [ ]:
def make_show_progress(exclamation):
    _inner = lambda epoch: print(f"{exclamation}! We have finished epoch {epoch}!")
    return _inner

In [ ]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We have finished epoch 0!
Nice!! We have finished epoch 1!
Nice!! We have finished epoch 2!
Nice!! We have finished epoch 3!
Nice!! We have finished epoch 4!


30

In [ ]:
from functools import partial

In [ ]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

## Callbacks as callable classes

In [ ]:
class ProgressShowingCallback:
    def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [ ]:
cb = ProgressShowingCallback("Just super")

In [ ]:
slow_calculation(cb)

Just super! We've finished epoch 0!
Just super! We've finished epoch 1!
Just super! We've finished epoch 2!
Just super! We've finished epoch 3!
Just super! We've finished epoch 4!


30

## Multiple callback funs, `*args` and `**kwargs`

In [ ]:
def f(*args, **kwargs): print(f"args: {args}; kwargs: {kwargs}")

In [ ]:
f(3, 'a', thing1='hello')

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [ ]:
def g(a, b, c=0): print(a, b, c)

In [ ]:
args = [1, 2]
kwargs = {'c': 3}
g(*args, **kwargs)

1 2 3


In [ ]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, val=res)
    return res

In [ ]:
class PrintStepCallback():
    def __init__(self): pass
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc(self, *args, **kwargs): print(f"Done step")
        

In [ ]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [ ]:
class PrintStatusCallback:
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
    def after_calc(self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [ ]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

## `__dunder__` thingies

Anything that looks like `__this__` is, in some way, special. Python, or some library, can define some functions that they wull call at certain documentated times. For instance, when your class is setting up a new object, python will call `__init__`. These are defined as part of the python data model

For instance, if python sees `+`, then it will call the special method `__add__`. If you try to display an object in Jupyter (or lots of other places in Python) it will call `__repr__`

In [ ]:
class SloppyAdder:
    def __init__(self, o): self.o = o
    def __add__(self, b): return SloppyAdder(self.o + b.o + 0.01)
    def __repr__(self): return str(self.o)

In [ ]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a + b

3.01

Special methods you should probably know about (see data model link above) are
- `__getitem__`
- `__getattr__`
- `__setattr__`
- `__del__`
- `__init__`
- `__new__`
- `__enter__`
- `__exit__`
- `__len__`
- `__repr__`
- `__str__`

### `__getattr__` and `getattr`

In [ ]:
class A: a, b = 1, 2

In [ ]:
a = A()

In [ ]:
a.b

2

In [ ]:
getattr(a, 'b' if random.random() > 0.5 else 'a')

2

In [ ]:
class B:
    a, b= 1, 2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f"Hello from {k}"

In [ ]:
b = B()

In [ ]:
b.a

1

In [ ]:
b.foo

'Hello from foo'